In [2]:
import pandas as pd
from copy import deepcopy

In [3]:
import os

if os.getcwd() != "../data/raw":
    os.chdir("../data/raw")
os.getcwd()

'/root/restaurants/data/raw'

In [4]:
visit_data = pd.read_csv("air_visit_data.csv")
visit_data["visit_date"] = pd.to_datetime(visit_data["visit_date"])
visit_data = visit_data.rename(columns={"air_store_id": "store_id"})
visit_data["visit_date"] = visit_data["visit_date"].dt.date
visit_data.head()

,store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [5]:
store_info = pd.read_csv("air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }
)

In [6]:
date_info = pd.read_csv("date_info.csv")
visit_data = visit_data.rename(columns={"visit_date": "date"})
visit_data["date"] = visit_data["date"].astype("string")

date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

In [7]:
data = pd.merge(visit_data, date_info, on="date", how="left")
data = pd.merge(data, store_info, on="store_id", how="left")
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data = data.sort_values(["store_id", "date"]).reset_index(drop=True)
data.head()

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,day
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,1
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,2
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,4
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,5
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,6


In [7]:
def add_lag(df, lag, column):
    column_name = "lag_" + column + "_" + str(lag)

    if column_name in df.columns:
        return df

    lag_data = []

    for (store_id, col_val), temp in df.groupby(["store_id", column]):
        temp_rolled = (
            temp[["date", "visitors"]]
            .rolling(f"{lag}D", on="date", min_periods=1)
            .mean()
        )
        temp_rolled["store_id"] = store_id
        temp_rolled[column_name] = temp_rolled["visitors"]
        temp_rolled = temp_rolled.drop(columns="visitors")

        lag_data.append(temp_rolled)

    lag_data = pd.concat(lag_data)

    return df.merge(lag_data, on=["store_id", "date"], how="left")

In [29]:
def get_area_genre_lag(df, lag, column):
    column_name = "lag_area_genre_" + column + "_" + str(lag)

    if column_name in df.columns:
        return df

    lag_data = []
    for (area_name, genre_name), area_genre_data in df.groupby(
        ["area_name", "genre_name"]
    ):
        area_genre_data = area_genre_data[
            ["area_name", "genre_name", "date", "visitors", column]
        ]
        area_genre_data = area_genre_data.sort_values("date")
        area_genre_data_mean = area_genre_data.groupby(by=["date"]).visitors.mean()
        area_genre_data = area_genre_data.drop(columns=["visitors"])
        area_genre_data = area_genre_data.merge(
            area_genre_data_mean, on=["date"], how="right"
        ).drop_duplicates()

        for col_val, temp in area_genre_data.groupby(column):
            temp_rolled = (
                temp[["date", "visitors"]]
                .rolling(f"{lag}D", on="date", min_periods=1)
                .mean()
            )
            temp_rolled["area_name"] = area_name
            temp_rolled["genre_name"] = genre_name
            temp_rolled[column_name] = temp_rolled["visitors"]
            temp_rolled = temp_rolled.drop(columns="visitors")

            lag_data.append(temp_rolled)

    lag_data = pd.concat(lag_data)

    return df.merge(lag_data, on=["area_name", "genre_name", "date"], how="left")


In [32]:
new_data = deepcopy(data)

In [37]:
new_data = deepcopy(data)

for lag in [21, 35, 63, 140, 280, 350]:
    new_data = get_area_genre_lag(new_data, lag, "day_of_week")
    new_data = get_area_genre_lag(new_data, lag, "holiday_flg")
    new_data = add_lag(new_data, lag, "day_of_week")
    new_data = add_lag(new_data, lag, "holiday_flg")

new_data

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,...,lag_day_of_week_140,lag_holiday_flg_140,lag_area_genre_day_of_week_280,lag_area_genre_holiday_flg_280,lag_day_of_week_280,lag_holiday_flg_280,lag_area_genre_day_of_week_350,lag_area_genre_holiday_flg_350,lag_day_of_week_350,lag_holiday_flg_350
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,35.000000,35.000000,25.000000,25.000000,35.000000,35.000000,25.000000,25.000000,35.000000,35.000000
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,9.000000,22.000000,16.500000,20.750000,9.000000,22.000000,16.500000,20.750000,9.000000,22.000000
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,20.000000,21.333333,14.000000,17.875000,20.000000,21.333333,14.000000,17.875000,20.000000,21.333333
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,25.000000,22.250000,20.333333,18.366667,25.000000,22.250000,20.333333,18.366667,25.000000,22.250000
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,29.000000,23.600000,14.750000,17.763889,29.000000,23.600000,14.750000,17.763889,29.000000,23.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_fff68b929994bfbd,2017-04-18,6,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,...,4.105263,4.952000,8.314583,9.583013,4.388889,5.135246,9.226667,10.576728,4.526316,5.122530
252104,air_fff68b929994bfbd,2017-04-19,2,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,...,4.650000,4.960000,8.933333,9.585897,4.666667,5.135246,10.331667,10.574215,4.634146,5.110236
252105,air_fff68b929994bfbd,2017-04-20,2,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,...,4.500000,4.880000,8.950000,9.593590,4.641026,5.098361,10.175000,10.579293,4.707317,5.098039
252106,air_fff68b929994bfbd,2017-04-21,4,Friday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,...,5.950000,4.880000,8.937500,9.581090,5.775000,5.102459,9.770000,10.550505,5.627907,5.093750


In [38]:
new_data.to_csv("../processed/data.csv", index=False)

In [8]:
data.head()

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,day
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,1
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,2
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,4
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,5
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,6


In [9]:

# df = deepcopy(data)
df = data[["visitors", "holiday_flg", "year", "month", "day", "store_id", "date"]].head(
    2000
)

# df[["day_of_week", "genre_name", "area_name"]] = df[["day_of_week", "genre_name", "area_name"]].apply(LabelEncoder().fit_transform)
y = data[["store_id", "visitors"]].head(2000).set_index("store_id").squeeze()
# X = extract_relevant_features(df, y, column_id='store_id', column_sort="date")
# X
y

store_id
air_00a91d42b08b08d9    35
air_00a91d42b08b08d9     9
air_00a91d42b08b08d9    20
air_00a91d42b08b08d9    25
air_00a91d42b08b08d9    29
                        ..
air_03963426c9312048    58
air_03963426c9312048     6
air_03963426c9312048    34
air_03963426c9312048    31
air_03963426c9312048    44
Name: visitors, Length: 2000, dtype: int64

In [15]:
from tsfresh import extract_features

extracted_features = extract_features(df, column_id="store_id", column_sort="date")

Feature Extraction: 100%|██████████| 20/20 [00:04<00:00,  4.91it/s]


In [16]:
extracted_features

,holiday_flg__variance_larger_than_standard_deviation,holiday_flg__has_duplicate_max,holiday_flg__has_duplicate_min,holiday_flg__has_duplicate,holiday_flg__sum_values,holiday_flg__abs_energy,holiday_flg__mean_abs_change,holiday_flg__mean_change,holiday_flg__mean_second_derivative_central,holiday_flg__median,...,month__fourier_entropy__bins_5,month__fourier_entropy__bins_10,month__fourier_entropy__bins_100,month__permutation_entropy__dimension_3__tau_1,month__permutation_entropy__dimension_4__tau_1,month__permutation_entropy__dimension_5__tau_1,month__permutation_entropy__dimension_6__tau_1,month__permutation_entropy__dimension_7__tau_1,month__query_similarity_count__query_None__threshold_0.0,month__mean_n_absolute_max__number_of_maxima_7
air_00a91d42b08b08d9,0.0,0.0,1.0,1.0,1.0,1.0,0.008658,0.000000,0.000000,0.0,...,0.147416,0.233472,0.573694,0.055945,0.084198,0.112641,0.141274,0.170101,NaN,12.000000
air_0164b9927d20bcc3,0.0,1.0,1.0,1.0,5.0,5.0,0.067568,0.000000,0.000000,0.0,...,0.141441,0.263667,0.762383,0.081410,0.122738,0.164491,0.206675,0.249296,NaN,12.000000
air_0241aa3964b7f861,0.0,1.0,1.0,1.0,17.0,17.0,0.068354,-0.002532,0.001269,0.0,...,0.125256,0.181214,0.440417,0.035400,0.053206,0.071083,0.089032,0.107053,NaN,12.000000
air_0328696196e46f18,0.0,1.0,1.0,1.0,6.0,6.0,0.086957,0.000000,0.000000,0.0,...,0.318391,0.403106,1.105582,0.100480,0.151699,0.203587,0.256156,0.309420,NaN,11.857143
air_034a3d5b40d5b1b1,0.0,1.0,1.0,1.0,15.0,15.0,0.088000,0.000000,0.000000,0.0,...,0.138673,0.219798,0.612541,0.052317,0.078718,0.105283,0.132014,0.158912,NaN,12.000000
air_036d4f1ee7285390,0.0,1.0,1.0,1.0,10.0,10.0,0.071429,0.000000,0.000000,0.0,...,0.136002,0.181214,0.451164,0.047510,0.071463,0.095549,0.119770,0.144126,NaN,12.000000
air_0382c794b73b51ad,0.0,1.0,1.0,1.0,17.0,17.0,0.087542,0.000000,0.000000,0.0,...,0.090729,0.136002,0.350689,0.045182,0.067951,0.090840,0.113850,0.136981,NaN,12.000000
air_03963426c9312048,0.0,1.0,1.0,1.0,15.0,15.0,0.079710,0.000000,0.000000,0.0,...,0.045395,0.045395,0.136002,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,NaN,10.000000


In [18]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y)
features_filtered

AssertionError: X and y must contain the same number of samples.